In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.misc import imread
import matplotlib.image as mpimg
import os
import tensorflow as tf
import keras.backend as K
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import load_model
import pandas as pd
from tensorflow.contrib.session_bundle import exporter
from keras.models import model_from_config

Using TensorFlow backend.


In [5]:
test_path = 'data/test-jpg-additional/'
test_names = []
image_size = 256
for i in range(20522):
    test_names.append('file_' + str(i))
print(len(test_names))

20522


In [6]:
def load_image(data_path, indices, image_size):
    num_images = 0
    dataset = np.ndarray(shape=(len(indices), image_size, image_size, 3),dtype=np.float32)
    for i in tqdm(range(len(indices)), total = len(indices)):
        image = data_path + indices[i] + '.jpg'
        image_data = mpimg.imread(image)[:,:,:3]
        #Normalization
        image_data = np.multiply(image_data, 1.0/255.0)
        dataset[num_images, :, :, :] = image_data
        num_images += 1
    print('Full dataset tensor:', dataset.shape)
    return dataset

In [7]:
x_test = load_image(test_path, test_names, image_size)

100%|████████████████████████████████████████████████████████████████████████████| 20522/20522 [06:54<00:00, 49.47it/s]


Full dataset tensor: (20522, 256, 256, 3)


In [18]:
num_classes = 17

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_test.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.001, decay=1e-6)
def accuracy_with_threshold(y_true, y_pred):
    threshold = 0.5
    y_pred = K.cast(K.greater(y_pred, threshold), K.floatx())
    return K.mean(K.equal(y_true, y_pred))

def hamming_dist(y_true, y_pred):
    threshold = 0.5
    y_pred = K.cast(K.greater(y_pred, threshold), K.floatx())
    return K.mean(K.sum(K.abs(y_true - y_pred), axis=1))

In [19]:
model.load_weights('planet1.h5')

In [20]:
pred = model.predict(x_test, batch_size=32, verbose=0)

In [21]:
y_pred = (pred >= 0.5).astype(np.int32)

In [22]:
with open('submission1/planet_predict_1_3.csv', 'w') as outfn:
    for idx in range(len(test_names)):
        s = test_names[idx]
        for j in range(num_classes):
            s += ', {:.3f}'.format(pred[idx,j])
        outfn.write(s)
        outfn.write('\n')